In [1]:
# Scaled variable
import pyvista
from ufl import as_tensor, as_vector
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.mesh import MeshTags, meshtags
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
from numpy import sin, cos
L = 1
W = 0.5
# mu = 1
rho = 1
delta = W / L
gamma = 0.4 * delta**2
# beta = 1.25
# lambda_ = beta
g = gamma

DOLFIN_EPS = 1e-9

from dolfinx import *

# xdmf = XDMFFile("output/density.xdmf")

ModuleNotFoundError: No module named 'pyvista'

In [ ]:
##################################### 模型创建 #########################################
msh = mesh.create_box(MPI.COMM_WORLD, [np.array([0, 0, 0]), np.array([L, W, W])],
                         [30, 15, 15], cell_type=mesh.CellType.hexahedron)




In [ ]:
##################################### 定义迪利克雷边界 ####################################
def clamped_boundary(x):
    return np.isclose(x[0], 0)
fdim = msh.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(msh, fdim, clamped_boundary)

##################################### 定义自然边界 #######################################
# boundary_facets.
# load_marker = mesh.MeshFunction()
def load_marker(x):
    return np.isclose(x[0], L)
load_facets = mesh.locate_entities_boundary(msh, fdim, load_marker)
mt = MeshTags(meshtags(msh, fdim, load_facets, np.full(load_facets.shape, 1)))


In [ ]:
# T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

In [ ]:
# ds = ufl.Measure("ds", domain=msh)

In [ ]:
############################################ 数据准备 ######################################
# 材料弹性矩阵
# E = 21
# mu = 0.3
# Cons = E/(1+mu)/(1-2*mu)
# Cijkl = Cons*as_tensor([[1-mu, mu, mu, 0, 0, 0],
#                         [mu, 1-mu, mu, 0, 0, 0],
#                         [mu, mu, 1-mu, 0, 0, 0],
#                         [1, 0, 0, (1-2*mu)/2, 0, 0],
#                         [0, 0, 0, 0, (1-2*mu)/2, 0],
#                         [0, 0, 0, 0, 0, (1-2*mu)/2]])

mE1 = 114e9
mE2 = 8.61e9
mE3 = 8.61e9
mMu12 = 0.3
mMu13 = 0.3
mMu23 = 0.45
mG12 = 4.16e9
mG13 = 4.16e9
mG23 = 3.0e9

Cons = (1-mMu12*mMu12-mMu23*mMu23-mMu13*mMu13-2*mMu12*mMu23*mMu13)/(mE1*mE2*mE3)
mC11 = (1-mMu23*mMu23)/(mE2*mE3*Cons)
mC22 = (1-mMu13*mMu13)/(mE1*mE3*Cons)
mC33 = (1-mMu12*mMu12)/(mE1*mE2*Cons)
mC12 = mC21 = (mMu12+mMu23*mMu13)/(mE1*mE3*Cons)
mC13 = mC31 = (mMu13+mMu12*mMu23)/(mE1*mE2*Cons)
mC23 = mC32 = (mMu23+mMu12*mMu13)/(mE1*mE2*Cons)
mC44 = mG23
mC55 = mG13
mC66 = mG12

# 正交各向异性材料弹性矩阵
Cijkl = Cons*as_tensor([[mC11, mC12, mC13,    0,    0,    0],
                        [mC21, mC22, mC23,    0,    0,    0],
                        [mC31, mC32, mC33,    0,    0,    0],
                        [   0,    0,    0, mC44,    0,    0],
                        [   0,    0,    0,    0, mC55,    0],
                        [   0,    0,    0,    0,    0, mC66]])

# 转轴公式
# 使用欧拉角表示坐标系的旋转 Z(r1)X(r2)Z(r3)
def RM(r1, r2, r3):
    r1 = r1*np.pi/180
    r2 = r2*np.pi/180
    r3 = r3*np.pi/180
    R = as_tensor([[ cos(r1)*cos(r3)-cos(r2)*sin(r1)*sin(r3),  cos(r2)*cos(r1)*sin(r3)+sin(r1)*cos(r3),  sin(r2)*sin(r3)],
                   [-cos(r1)*sin(r3)-cos(r2)*sin(r1)*cos(r3),  cos(r2)*cos(r1)*cos(r3)-sin(r1)*sin(r3),  sin(r2)*cos(r3)],
                   [                         sin(r2)*sin(r1),                         -sin(r2)*cos(r1),          cos(r2)]])
    return R

def M(R):
    return as_tensor([[R[0,0]**2, R[1,0]**2, R[2,0]**2, 2*R[0,0]*R[1,0], 2*R[1,0]*R[2,0], 2*R[2,0]*R[0,0]],
                      [R[0,1]**2, R[1,1]**2, R[2,1]**2, 2*R[0,1]*R[1,1], 2*R[1,1]*R[2,1], 2*R[2,1]*R[0,1]],
                      [R[0,2]**2, R[1,2]**2, R[2,2]**2, 2*R[0,2]*R[1,2], 2*R[1,2]*R[2,2], 2*R[2,2]*R[0,2]],
                      [R[0,0]*R[0,1], R[1,0]*R[1,1], R[2,0]*R[2,1], R[0,0]*R[1,1]+R[0,1]*R[1,0], R[1,0]*R[2,1]+R[2,0]*R[1,1], R[2,0]*R[0,1]+R[2,1]*R[0,0]],
                      [R[0,1]*R[0,2], R[1,1]*R[1,2], R[2,2]*R[2,1], R[1,2]*R[0,1]+R[0,2]*R[1,1], R[1,1]*R[2,2]+R[2,1]*R[1,2], R[2,1]*R[0,2]+R[0,1]*R[2,2]],
                      [R[0,0]*R[0,2], R[1,2]*R[1,0], R[2,2]*R[2,0], R[0,2]*R[1,0]+R[0,0]*R[1,2], R[1,2]*R[2,0]+R[1,0]*R[2,2], R[2,2]*R[0,0]+R[2,0]*R[0,2]]])
                
def N(R):
    return as_tensor([[R[0,0]**2, R[1,0]**2, R[2,0]**2, R[0,0]*R[1,0], R[1,0]*R[2,0], R[2,0]*R[0,0]],
                      [R[0,1]**2, R[1,1]**2, R[2,1]**2, R[0,1]*R[1,1], R[1,1]*R[2,1], R[2,1]*R[0,1]],
                      [R[0,2]**2, R[1,2]**2, R[2,2]**2, R[0,2]*R[1,2], R[1,2]*R[2,2], R[2,2]*R[0,2]],
                      [2*R[0,0]*R[0,1], 2*R[1,0]*R[1,1], 2*R[2,0]*R[2,1], R[0,0]*R[1,1]+R[0,1]*R[1,0], R[1,0]*R[2,1]+R[2,0]*R[1,1], R[2,0]*R[0,1]+R[2,1]*R[0,0]],
                      [2*R[0,1]*R[0,2], 2*R[1,1]*R[1,2], 2*R[2,2]*R[2,1], R[1,2]*R[0,1]+R[0,2]*R[1,1], R[1,1]*R[2,2]+R[2,1]*R[1,2], R[2,1]*R[0,2]+R[0,1]*R[2,2]],
                      [2*R[0,0]*R[0,2], 2*R[1,2]*R[1,0], 2*R[2,2]*R[2,0], R[0,2]*R[1,0]+R[0,0]*R[1,2], R[1,2]*R[2,0]+R[1,0]*R[2,2], R[2,2]*R[0,0]+R[2,0]*R[0,2]]])

def CR(C, r1, r2, r3):
    return M(r1, r2, r3) * C * ufl.inv(N(r1, r2, r3))

# 应变voigt法表示
def strain3voigt(tensorStrain):
    return as_vector([tensorStrain[0,0],
                      tensorStrain[1,1],
                      tensorStrain[2,2],
                      2*tensorStrain[1,2],
                      2*tensorStrain[2,0],
                      2*tensorStrain[0,1]])

# voigt应力tensor表示
def stress3tensor(voigtVector):
    return as_tensor([[voigtVector[0], voigtVector[5], voigtVector[4]],
                      [voigtVector[5], voigtVector[1], voigtVector[3]],
                      [voigtVector[4], voigtVector[3], voigtVector[2]]])

# 材料应变
def strain(u):
    # return 1.0/2.0*(ufl.grad(u) + ufl.grad(u).T)
    return ufl.sym(ufl.grad(u))

# 材料应力
def stress(u, R1, R2, R3):
    # sts = ufl.dot(CR(R1, R2, R3), strain3voigt(strain(u)))
    sts = ufl.dot(CR(Cijkl, R1, R2, R3), strain3voigt(strain(u)))
    return stress3tensor(sts)


In [ ]:
# 定义有限元空间
####################################### 定义函数空间 #################################################
U = fem.VectorFunctionSpace(msh, ("Lagrange", 1, (msh.geometry.dim, )))
ANG_ROT_Z = fem.FunctionSpace(msh, ("DG", 1, (msh.geometry.dim, )))

# 试函数，任意函数
u = ufl.TrialFunction(U)
# 测试函数 与mesh维度相同
v = ufl.TestFunction(U)
u_sol, ang_old, ang = fem.Function(U), fem.Function(ANG_ROT_Z), fem.Function(ANG_ROT_Z, name="rotate_angle_Z")

######################################### 边界和载荷 ###############################################
u_D = np.array([0, 0, 0], dtype=default_scalar_type)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(U, fdim, boundary_facets), U)


f = fem.Constant(msh, default_scalar_type((0, 0, -10)))
# rhs = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) * ds(1)
# rhs = ufl.inner(f, v) * ufl.ds(domain=msh, subdomain_data=mt)
rhs = ufl.inner(f, v) * ufl.dx
# a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
lhs = ufl.inner(stress(u, 0, 0, 0), strain(v)) * ufl.dx


In [ ]:
problem = LinearProblem(lhs, rhs, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [ ]:
# pyvista.start_xvfb()

# # Create plotter and pyvista grid
# p = pyvista.Plotter()
# topology, cell_types, geometry = plot.vtk_mesh(msh)
# grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)

# # Attach vector values to grid and warp grid by vector
# grid["u"] = uh.x.array.reshape((geometry.shape[0], 3))
# actor_0 = p.add_mesh(grid, style="wireframe", color="k")
# warped = grid.warp_by_vector("u", factor=1.5)
# actor_1 = p.add_mesh(warped, show_edges=True)
# p.show_axes()
# p.show()
# if not pyvista.OFF_SCREEN:
#     p.show()
# else:
#     figure_as_array = p.screenshot("deflection.png")